In [24]:
!pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=c9442b4b2244bd254fc4e3f77c879e21f853be8e3520ba69a3af55b6484fabdf
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta


In [25]:
####PREPARATION OF ENHANCED DATASET WITH ALL TECHNICAL INDICATORS ON TOP OF FEW DATA COLUMNS
import numpy as np
import pandas as pd
import pandas_ta as ta

# Load the original dataset
file_path = "OriginalDataset.csv"
df = pd.read_csv(file_path)

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")

# Convert numerical columns to float after stripping any special characters
for col in ['Price', 'Open', 'High', 'Low']:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

# Convert 'Change %' to float after removing the '%' symbol
df['Change %'] = df['Change %'].astype(str).str.replace('%', '').astype(float) / 100

# Handle missing values in 'Vol.'
df['Vol.'] = df['Vol.'].astype(str).str.replace('K', 'e3').str.replace('M', 'e6')
df['Vol.'] = pd.to_numeric(df['Vol.'], errors='coerce')
df['Vol.'] = df['Vol.'].fillna(df['Vol.'].median())

# Sort by Date and Type before computing indicators
df = df.sort_values(by=["Type", "Date"])

# Compute Moving Averages (SMA, EMA) and Daily Returns
df['SMA_10'] = df.groupby("Type")['Price'].transform(lambda x: x.rolling(window=10).mean())
df['EMA_10'] = df.groupby("Type")['Price'].transform(lambda x: x.ewm(span=10, adjust=False).mean())
df['Daily Return'] = df.groupby("Type")['Price'].transform(lambda x: x.pct_change())

# Compute 'Price Change %'
df['Price Change %'] = df.groupby("Type")['Price'].pct_change()
df['Price Change %'] = df['Price Change %'].fillna(0)  # Fill initial NaNs

print("\n📌 Available Columns Before Adding Technical Indicators:", df.columns.tolist())

# Compute Advanced Technical Indicators
# RSI (Relative Strength Index)
df['RSI_14'] = df.groupby("Type", group_keys=False)['Price'].apply(lambda x: ta.rsi(x, length=14))

# MACD (Moving Average Convergence Divergence)
macd_indicators = df.groupby("Type", group_keys=False)['Price'].apply(lambda x: ta.macd(x, fast=12, slow=26, signal=9))
macd_df = macd_indicators.apply(pd.Series).reset_index(drop=True)
df = df.reset_index(drop=True)  # Ensure index consistency
df[['MACD', 'MACD_Signal', 'MACD_Hist']] = macd_df[['MACD_12_26_9', 'MACDs_12_26_9', 'MACDh_12_26_9']]

# Bollinger Bands (BB)
bb_indicators = df.groupby("Type", group_keys=False)['Price'].apply(lambda x: ta.bbands(x, length=20))
bb_df = bb_indicators.apply(pd.Series).reset_index(drop=True)
df[['Upper_BB', 'Middle_BB', 'Lower_BB']] = bb_df[['BBU_20_2.0', 'BBM_20_2.0', 'BBL_20_2.0']]

# ATR (Average True Range)
atr_values = df.groupby("Type", group_keys=False).apply(lambda x: ta.atr(high=x['High'], low=x['Low'], close=x['Price'], length=14)).reset_index(drop=True)
df['ATR'] = atr_values  # Assign ATR correctly

print("\n✅ Advanced Technical Indicators Added.")

# Fix Threshold Calculation for Trend Classification
def calculate_thresholds(asset_df):
    """Calculate Q1 (25th percentile) and Q3 (75th percentile) for thresholding trends."""
    quantiles = asset_df['Price Change %'].quantile([0.25, 0.75])
    return pd.Series({'Stable Lower': quantiles[0.25], 'Stable Upper': quantiles[0.75]})

# Compute thresholds per asset type
thresholds = df.groupby("Type", group_keys=False)[['Price Change %']].apply(calculate_thresholds).reset_index()

# Merge thresholds with dataset
df = df.merge(thresholds, on=["Type"], how="left")

# Assign Trend Labels
df['Trend'] = df.apply(lambda row: 1 if row['Price Change %'] > row['Stable Upper']
                                  else (-1 if row['Price Change %'] < row['Stable Lower'] else 0), axis=1)

# Fill NaNs using forward-fill and backward-fill
df = df.groupby("Type").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

# Save the processed dataset
df.to_csv("Enhanced_Dataset_with_Indicators.csv", index=False)

print("\n Final Dataset Saved as `Enhanced_Dataset_with_Indicators.csv`")

# Display dataset summary and first few rows
df.info()
print("\n🔍 First 5 Rows:")
print(df.head())

#  Check final column list
print("\n📌 Available Columns After Adding Technical Indicators:", df.columns.tolist())



📌 Available Columns Before Adding Technical Indicators: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'Type', 'SMA_10', 'EMA_10', 'Daily Return', 'Price Change %']

✅ Advanced Technical Indicators Added.


<ipython-input-25-13387ae381f1>:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  atr_values = df.groupby("Type", group_keys=False).apply(lambda x: ta.atr(high=x['High'], low=x['Low'], close=x['Price'], length=14)).reset_index(drop=True)
<ipython-input-25-13387ae381f1>:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Type").apply(lambda group: group.ffill().bfill()).reset_index(drop=True)



 Final Dataset Saved as `Enhanced_Dataset_with_Indicators.csv`
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3189 entries, 0 to 3188
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            3189 non-null   datetime64[ns]
 1   Price           3189 non-null   float64       
 2   Open            3189 non-null   float64       
 3   High            3189 non-null   float64       
 4   Low             3189 non-null   float64       
 5   Vol.            3189 non-null   float64       
 6   Change %        3189 non-null   float64       
 7   Type            3189 non-null   object        
 8   SMA_10          3189 non-null   float64       
 9   EMA_10          3189 non-null   float64       
 10  Daily Return    3189 non-null   float64       
 11  Price Change %  3189 non-null   float64       
 12  RSI_14          3189 non-null   float64       
 13  MACD            3189 non-null   float64     

In [27]:
#PREPARATION OF EXTERNAL FEATURES like STOCK INDEX OF US MARKETS
import pandas as pd

# Load the Stock Index dataset
stock_index = pd.read_csv("Stock_Index_Cleaned.csv")

# Convert Date column to proper datetime format and remove timezone
stock_index['Date'] = pd.to_datetime(stock_index['Date'], utc=True, errors='coerce')
stock_index['Date'] = stock_index['Date'].dt.tz_localize(None)  # Remove timezone
stock_index['Date'] = stock_index['Date'].dt.normalize()  # Remove time component

#  Round 'SP500_Close' to 2 decimal places
stock_index['SP500_Close'] = stock_index['SP500_Close'].round(2)

#  Save the cleaned dataset
stock_index.to_csv("Stock_Index_Cleaned.csv", index=False)

#  Display first few rows to confirm changes
print("\n Fixed & Rounded Stock Index Sample:\n", stock_index.head())


 Fixed & Rounded Stock Index Sample:
         Date  SP500_Close
0 2021-01-04      3700.65
1 2021-01-05      3726.86
2 2021-01-06      3748.14
3 2021-01-07      3803.79
4 2021-01-08      3824.68


In [28]:
#Checking date format of each additional dataset

# Load main dataset
df = pd.read_csv("Enhanced_Dataset_with_Indicators.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Ensure Date is in datetime format

# Load external datasets
interest_rates = pd.read_csv("Interest_Rates.csv")
oil_prices = pd.read_csv("WTIPrice.csv")
stock_index = pd.read_csv("Stock_Index_Cleaned.csv")
gas_storage = pd.read_csv("Gas_Storage.csv")

# Convert Date columns to datetime format for merging
interest_rates['Date'] = pd.to_datetime(interest_rates['Date'])
oil_prices['Date'] = pd.to_datetime(oil_prices['Date'])

# Fix the stock index date format (remove timezone information)
#stock_index['Date'] = pd.to_datetime(stock_index['Date'], utc=True).dt.tz_localize(None)
stock_index['Date'] = pd.to_datetime(stock_index['Date'])

gas_storage['Date'] = pd.to_datetime(gas_storage['Date'])

# Display dataset samples
print("\n📌 Interest Rates Sample:\n", interest_rates.head())
print("\n📌 Oil Prices Sample:\n", oil_prices.head())
print("\n📌 Stock Index Sample:\n", stock_index.head())
print("\n📌 Gas Storage Sample:\n", gas_storage.head())


📌 Interest Rates Sample:
         Date  InterestRate
0 2021-01-01          0.09
1 2021-02-01          0.08
2 2021-03-01          0.07
3 2021-04-01          0.07
4 2021-05-01          0.06

📌 Oil Prices Sample:
         Date  Price
0 2021-01-04  47.47
1 2021-01-05  49.78
2 2021-01-06  50.45
3 2021-01-07  50.63
4 2021-01-08  52.14

📌 Stock Index Sample:
         Date  SP500_Close
0 2021-01-04      3700.65
1 2021-01-05      3726.86
2 2021-01-06      3748.14
3 2021-01-07      3803.79
4 2021-01-08      3824.68

📌 Gas Storage Sample:
         Date  GasStorage_in_BCF
0 2021-01-01               3330
1 2021-01-08               3196
2 2021-01-15               3009
3 2021-01-22               2881
4 2021-01-29               2689


In [29]:
#merging onto original dataset with interestrates data

# Load the enhanced dataset
df = pd.read_csv("Enhanced_Dataset_with_Indicators.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Convert Date column
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month  # Extract Year and Month

# Load Interest Rates Data
interest_rates = pd.read_csv("Interest_Rates.csv")
interest_rates['Date'] = pd.to_datetime(interest_rates['Date'])  # Convert Date column
interest_rates['Year'] = interest_rates['Date'].dt.year
interest_rates['Month'] = interest_rates['Date'].dt.month  # Extract Year and Month

# 📌 Merge Interest Rates correctly on Year & Month
df = df.merge(interest_rates[['Year', 'Month', 'InterestRate']], on=['Year', 'Month'], how="left")

# ✅ Drop unnecessary columns (Year & Month) after merging
df.drop(columns=['Year', 'Month'], inplace=True)

# ✅ Save the merged dataset
df.to_csv("Enhanced_Dataset_with_InterestRates.csv", index=False)

# ✅ Display dataset summary
print("\n✅ Merged Interest Rates Successfully!")
df.info()
print("\n🔍 First 5 Rows with Interest Rates:")
print(df.head())

# ✅ Check final columns
print("\n📌 Final Columns After Merging Interest Rates:", df.columns.tolist())



✅ Merged Interest Rates Successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3189 entries, 0 to 3188
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            3189 non-null   datetime64[ns]
 1   Price           3189 non-null   float64       
 2   Open            3189 non-null   float64       
 3   High            3189 non-null   float64       
 4   Low             3189 non-null   float64       
 5   Vol.            3189 non-null   float64       
 6   Change %        3189 non-null   float64       
 7   Type            3189 non-null   object        
 8   SMA_10          3189 non-null   float64       
 9   EMA_10          3189 non-null   float64       
 10  Daily Return    3189 non-null   float64       
 11  Price Change %  3189 non-null   float64       
 12  RSI_14          3189 non-null   float64       
 13  MACD            3189 non-null   float64       
 14  MACD_Signal     3

In [30]:
#merging onto original dataset with WTI OILPRICE data

# ✅ Load the latest enhanced dataset (already merged with Interest Rates)
df = pd.read_csv("Enhanced_Dataset_with_InterestRates.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Convert Date to datetime format

# ✅ Load WTI Oil Prices dataset
oil_prices = pd.read_csv("WTIPrice.csv")
oil_prices['Date'] = pd.to_datetime(oil_prices['Date'])  # Convert Date to datetime

# ✅ Rename column for clarity
oil_prices = oil_prices.rename(columns={"Price": "WTI_Oil_Price"})

# ✅ Merge with the main dataset on 'Date'
df = df.merge(oil_prices, on="Date", how="left")

# ✅ Forward-fill missing WTI prices for **all asset classes**
df['WTI_Oil_Price'] = df.groupby("Type")['WTI_Oil_Price'].ffill()

# ✅ Save the updated dataset (KEEPING the same structure)
df.to_csv("Enhanced_Dataset_with_IntRatesWTIPrice.csv", index=False)

# ✅ Display final check
print("\n✅ WTI Oil Prices Merged! Updated Dataset Sample:")
print(df.head())



✅ WTI Oil Prices Merged! Updated Dataset Sample:
        Date   Price    Open    High     Low      Vol.  Change %  \
0 2021-01-04  1946.6  1908.2  1948.7  1906.1  273370.0    0.0272   
1 2021-01-05  1954.4  1946.0  1957.0  1938.4  201950.0    0.0040   
2 2021-01-06  1908.6  1952.8  1962.5  1902.6  366610.0   -0.0234   
3 2021-01-07  1913.6  1921.5  1929.6  1907.5  199980.0    0.0026   
4 2021-01-08  1835.4  1915.2  1918.4  1827.8  443510.0   -0.0409   

           Type   SMA_10       EMA_10  ...  MACD_Hist     Upper_BB  Middle_BB  \
0  GOLD FUTURES  1878.98  1946.600000  ...   6.846452  1935.427938    1864.91   
1  GOLD FUTURES  1878.98  1948.018182  ...   5.795011  1935.427938    1864.91   
2  GOLD FUTURES  1878.98  1940.851240  ...   7.187079  1935.427938    1864.91   
3  GOLD FUTURES  1878.98  1935.896469  ...   8.471462  1935.427938    1864.91   
4  GOLD FUTURES  1878.98  1917.624384  ...  12.363844  1935.427938    1864.91   

      Lower_BB        ATR  Stable Lower  Stable Upper 

In [31]:
#merging onto original dataset with StockIndex data

# ✅ Load the latest enhanced dataset (already merged with Interest Rates & WTI Oil Prices)
df = pd.read_csv("Enhanced_Dataset_with_IntRatesWTIPrice.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Ensure Date is in datetime format

# ✅ Load Stock Index dataset
stock_index = pd.read_csv("Stock_Index_Cleaned.csv")
stock_index['Date'] = pd.to_datetime(stock_index['Date'])  # Convert Date to datetime format

# ✅ Rename column for clarity
stock_index = stock_index.rename(columns={"SP500_Close": "SP500_Index"})

# ✅ Merge with the main dataset on 'Date'
df = df.merge(stock_index, on="Date", how="left")

# ✅ Forward-fill missing Stock Index values for **each asset class**
df['SP500_Index'] = df.groupby("Type")['SP500_Index'].ffill()

# ✅ Save the updated dataset (KEEPING the same structure)
df.to_csv("Enhanced_Dataset_with_IntRatesWTIStockIndex.csv", index=False)

# ✅ Display final check for null values in SP500_Index
null_counts = df['SP500_Index'].isna().sum()
df.head(), null_counts


(        Date   Price    Open    High     Low      Vol.  Change %  \
 0 2021-01-04  1946.6  1908.2  1948.7  1906.1  273370.0    0.0272   
 1 2021-01-05  1954.4  1946.0  1957.0  1938.4  201950.0    0.0040   
 2 2021-01-06  1908.6  1952.8  1962.5  1902.6  366610.0   -0.0234   
 3 2021-01-07  1913.6  1921.5  1929.6  1907.5  199980.0    0.0026   
 4 2021-01-08  1835.4  1915.2  1918.4  1827.8  443510.0   -0.0409   
 
            Type   SMA_10       EMA_10  ...     Upper_BB  Middle_BB  \
 0  GOLD FUTURES  1878.98  1946.600000  ...  1935.427938    1864.91   
 1  GOLD FUTURES  1878.98  1948.018182  ...  1935.427938    1864.91   
 2  GOLD FUTURES  1878.98  1940.851240  ...  1935.427938    1864.91   
 3  GOLD FUTURES  1878.98  1935.896469  ...  1935.427938    1864.91   
 4  GOLD FUTURES  1878.98  1917.624384  ...  1935.427938    1864.91   
 
       Lower_BB        ATR  Stable Lower  Stable Upper  Trend  InterestRate  \
 0  1794.392062  36.568068     -0.004384       0.00586      0          0.09  

In [32]:
#merging onto original dataset with gas storage data

# ✅ Load the latest dataset with WTI Oil Prices & Interest Rates
df = pd.read_csv("Enhanced_Dataset_with_IntRatesWTIStockIndex.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Convert Date to datetime format

# ✅ Load Gas Storage dataset
gas_storage = pd.read_csv("Gas_Storage.csv")
gas_storage['Date'] = pd.to_datetime(gas_storage['Date'])  # Convert Date to datetime format

# ✅ Merge with the main dataset on 'Date' (Left Join)
df = df.merge(gas_storage, on="Date", how="left")

# ✅ Apply Gas Storage Values **ONLY for NATURALGAS FUTURES**
df['GasStorage_in_BCF'] = df.apply(
    lambda row: row['GasStorage_in_BCF'] if row['Type'] == "NATURALGAS FUTURES" else None, axis=1
)

# ✅ Forward-Fill Gas Storage Values for Natural Gas ONLY
df['GasStorage_in_BCF'] = df.groupby("Type")['GasStorage_in_BCF'].ffill()

# ✅ Save the corrected dataset
df.to_csv("Final_Enhanced_Dataset.csv", index=False)

# ✅ Verify Gold & Silver have NaN values for Gas Storage
print("\n🔍 Unique Gas Storage Values in Gold & Silver:")
print(df[df["Type"].isin(["GOLD FUTURES", "SILVER FUTURES"])]['GasStorage_in_BCF'].unique())

print("\n✅ Gas Storage **ONLY** Updated for Natural Gas Futures!")



🔍 Unique Gas Storage Values in Gold & Silver:
[nan]

✅ Gas Storage **ONLY** Updated for Natural Gas Futures!


In [33]:
#Setting Final Dataset with 4 missing values of NGASFUTURES to actual ones

# ✅ Load the final dataset
df = pd.read_csv("Final_Enhanced_Dataset.csv")
df['Date'] = pd.to_datetime(df['Date'])  # Convert Date to datetime format

# ✅ Identify the missing values
missing_gas_storage = df[(df["Type"] == "NATURALGAS FUTURES") & (df["GasStorage_in_BCF"].isna())]
print("\n🔍 Missing Gas Storage Entries Before Fix:\n", missing_gas_storage)

# ✅ Fill the remaining NaN values with 3330 **ONLY for NATURALGAS FUTURES**
df.loc[(df["Type"] == "NATURALGAS FUTURES") & (df["GasStorage_in_BCF"].isna()), "GasStorage_in_BCF"] = 3330

# ✅ Save the updated dataset
df.to_csv("Final_Enhanced_Dataset.csv", index=False)

# ✅ Verify that no missing values remain
print("\n✅ Missing Gas Storage Entries After Fix:\n", df["GasStorage_in_BCF"].isna().sum())
print("\n✅ Updated Dataset Saved as `Final_Enhanced_Dataset.csv`")


🔍 Missing Gas Storage Entries Before Fix:
            Date  Price   Open   High    Low     Vol.  Change %  \
1050 2021-01-04  2.561  2.601  2.643  2.553  69270.0    0.0139   
1051 2021-01-05  2.673  2.577  2.698  2.571  85700.0    0.0437   
1052 2021-01-06  2.684  2.658  2.732  2.584  69450.0    0.0041   
1053 2021-01-07  2.691  2.691  2.719  2.635  74220.0    0.0026   

                    Type  SMA_10    EMA_10  ...  Middle_BB  Lower_BB  \
1050  NATURALGAS FUTURES   2.669  2.561000  ...    2.62965  2.481996   
1051  NATURALGAS FUTURES   2.669  2.581364  ...    2.62965  2.481996   
1052  NATURALGAS FUTURES   2.669  2.600025  ...    2.62965  2.481996   
1053  NATURALGAS FUTURES   2.669  2.616566  ...    2.62965  2.481996   

           ATR  Stable Lower  Stable Upper  Trend  InterestRate  \
1050  0.112083     -0.023853      0.024149      0          0.09   
1051  0.112083     -0.023853      0.024149      1          0.09   
1052  0.112083     -0.023853      0.024149      0          0.09

In [34]:
#Rounding off numerical columns in final dataset

# Load the dataset
file_path = "Final_Enhanced_Dataset.csv"
df = pd.read_csv(file_path)

# Round all numeric columns to 2 decimal places
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].round(2)

# Save the updated dataset
updated_file_path = "Final_Enhanced_Dataset.csv"
df.to_csv(updated_file_path, index=False)
df.head()
# Display the first few rows to verify rounding

,Date,Price,Open,High,Low,Vol.,Change %,Type,SMA_10,EMA_10,...,Middle_BB,Lower_BB,ATR,Stable Lower,Stable Upper,Trend,InterestRate,WTI_Oil_Price,SP500_Index,GasStorage_in_BCF
0,2021-01-04,1946.6,1908.2,1948.7,1906.1,273370.0,0.03,GOLD FUTURES,1878.98,1946.60,...,1864.91,1794.39,36.57,-0.0,0.01,0,0.09,47.47,3700.65,NaN
1,2021-01-05,1954.4,1946.0,1957.0,1938.4,201950.0,0.00,GOLD FUTURES,1878.98,1948.02,...,1864.91,1794.39,36.57,-0.0,0.01,0,0.09,49.78,3726.86,NaN
2,2021-01-06,1908.6,1952.8,1962.5,1902.6,366610.0,-0.02,GOLD FUTURES,1878.98,1940.85,...,1864.91,1794.39,36.57,-0.0,0.01,-1,0.09,50.45,3748.14,NaN
3,2021-01-07,1913.6,1921.5,1929.6,1907.5,199980.0,0.00,GOLD FUTURES,1878.98,1935.90,...,1864.91,1794.39,36.57,-0.0,0.01,0,0.09,50.63,3803.79,NaN
4,2021-01-08,1835.4,1915.2,1918.4,1827.8,443510.0,-0.04,GOLD FUTURES,1878.98,1917.62,...,1864.91,1794.39,36.57,-0.0,0.01,-1,0.09,52.14,3824.68,NaN


In [ ]:
##EDA PLOTS


# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the final dataset
file_path = "Final_Enhanced_Dataset.csv"  # Change this if needed
df = pd.read_csv(file_path)

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Round off all numerical columns to 2 decimal places
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].round(2)

# Display summary statistics
print("\n📌 Summary Statistics:")
print(df.describe())

# Check for missing values
print("\n📌 Missing Values:")
print(df.isnull().sum())

# Unique asset types
print("\n📌 Asset Types:", df['Type'].unique())

# ------------------------------ #
# **1️⃣ Correlation Heatmaps (Per Asset Type)**
# ------------------------------ #

for asset in df['Type'].unique():
    plt.figure(figsize=(10, 6))
    asset_df = df[df['Type'] == asset].select_dtypes(include=['float64', 'int64'])
    sns.heatmap(asset_df.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
    plt.title(f"🔹 Correlation Heatmap - {asset}")
    plt.show()

# ------------------------------ #
# **2️⃣ Feature Distributions**
# ------------------------------ #

features_to_plot = ["Price", "Daily Return", "RSI_14", "MACD", "ATR", "InterestRate", "WTI_Oil_Price"]

for feature in features_to_plot:
    plt.figure(figsize=(10, 4))
    sns.histplot(df[feature], kde=True, bins=30, color='blue', edgecolor='black')
    plt.title(f"🔹 Distribution of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

# ------------------------------ #
# **3️⃣ Time-Series Price Trends**
# ------------------------------ #

for asset in df['Type'].unique():
    plt.figure(figsize=(12, 5))
    asset_df = df[df['Type'] == asset]
    plt.plot(asset_df['Date'], asset_df['Price'], label=f"{asset} Price", color="blue")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(f"🔹 Price Trend - {asset}")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()

# ------------------------------ #
# **4️⃣ Boxplot of Volatility (ATR)**
# ------------------------------ #

plt.figure(figsize=(12, 6))
sns.boxplot(x="Type", y="ATR", data=df, palette="coolwarm")
plt.title("🔹 Volatility Comparison (ATR) Across Asset Types")
plt.xlabel("Asset Type")
plt.ylabel("ATR (Average True Range)")
plt.grid(True)
plt.show()

# ------------------------------ #
# **5️⃣ Scatter Plots - Key Relationships**
# ------------------------------ #

scatter_features = [("InterestRate", "Price"), ("WTI_Oil_Price", "Price"), ("RSI_14", "Daily Return")]

for x_feat, y_feat in scatter_features:
    plt.figure(figsize=(10, 5))
    sns.scatterplot(x=df[x_feat], y=df[y_feat], hue=df["Type"], palette="coolwarm")
    plt.xlabel(x_feat)
    plt.ylabel(y_feat)
    plt.title(f"🔹 Scatter Plot: {x_feat} vs {y_feat}")
    plt.grid(True)
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
file_path = "Final_Enhanced_Dataset.csv"
df = pd.read_csv(file_path)

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Define rolling window size
rolling_window = 90

# Define asset types
assets = df['Type'].unique()

# Create rolling correlation plots
for asset in assets:
    df_asset = df[df['Type'] == asset].set_index('Date')

    plt.figure(figsize=(12, 6))
    plt.title(f"Rolling 90-Day Correlations - {asset}")

    # Price vs Interest Rate
    df_asset['Price'].rolling(rolling_window).corr(df_asset['InterestRate']).plot(label='Price vs Interest Rate', linestyle='--')

    # Price vs WTI Oil Price
    df_asset['Price'].rolling(rolling_window).corr(df_asset['WTI_Oil_Price']).plot(label='Price vs WTI Oil Price', linestyle='-.')

    # Price vs SP500 Index
    df_asset['Price'].rolling(rolling_window).corr(df_asset['SP500_Index']).plot(label='Price vs SP500 Index', linestyle=':')

    # Price vs Gas Storage (only for Natural Gas Futures)
    if asset == "NATURALGAS FUTURES":
        df_asset['Price'].rolling(rolling_window).corr(df_asset['GasStorage_in_BCF']).plot(label='Price vs Gas Storage', linestyle='-')

    plt.legend()
    plt.ylabel("Correlation")
    plt.xlabel("Date")
    plt.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    plt.grid(True)
    plt.show()
